### Ver5 Enhanced with Spectral Attention and Multi-Objective Loss

Update di versi 4 :
- Ada Spectral Attention
- Ada Multi-Objective Loss
- Ada mekanisme early stopping
- Beberapa peningkatan di proses evaluasi
- Perbaikan data splitting dll, class-balanced sampling, disable boundary loss, focal gamma 4.0, reduced dropout, warmup LR scheduler, fixed colormap loading

In [ ]:
# ===========================================
# Cell 1. Import Library dan Setup Environment
# ===========================================

import os
import json
import random
from pathlib import Path
from collections import Counter, defaultdict

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
from sklearn.preprocessing import MinMaxScaler
from scipy.ndimage import distance_transform_edt

# Gunakan GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device yang digunakan:", device)

# Monitor GPU memory
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Total Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

In [ ]:
# ===========================================
# Cell 2. Fungsi Bantuan Umum + FIXED Colormap Loader
# ===========================================

from matplotlib.colors import ListedColormap

def seed_everything(seed=42):
    """Menetapkan seed random agar hasil eksperimen bisa direplikasi"""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

seed_everything(42)


def load_colormap_from_json(json_path, label_remap, num_classes):
    """
    FIXED: Load colormap dari label_classes.json dengan mapping yang benar
    
    Args:
        json_path: Path ke label_classes.json
        label_remap: Dictionary {original_label: new_index}
        num_classes: Jumlah kelas setelah remapping
    
    Returns:
        ListedColormap dengan warna yang sesuai
    """
    if not os.path.exists(json_path):
        print("[WARNING] label_classes.json not found! Using default colors")
        return ListedColormap(['black'] + [plt.cm.tab20(i) for i in range(num_classes-1)])
    
    with open(json_path, 'r') as f:
        label_info = json.load(f)
    
    # Reverse mapping: new_idx -> orig_label
    reverse_map = {v: k for k, v in label_remap.items()}
    
    # Mapping nama kelas ke warna (berdasarkan original dataset indices)
    # Original indices: {0, 8, 12, 13, 14, 18, 38}
    name_to_color = {}
    for cls_info in label_info:
        name_to_color[cls_info['name']] = cls_info['color'][:7]
    
    # Build color list
    colors = ['#000000']  # Background (index 0) always black
    
    # Manual mapping berdasarkan data Anda:
    # 0 → 0: Deep water (hitam)
    # 8 → 1: Sand (coklat muda)
    # 12 → 2: Kelp (coklat)
    # 13 → 3: Laminaria hyperborea (ungu)
    # 14 → 4: Laminaria digitata (biru)
    # 18 → 5: Rockweed (hijau)
    # 38 → 6: Trawl track (merah)
    
    class_mapping = {
        0: 'Deep water',
        8: 'Sand',
        12: 'Kelp',
        13: 'Laminaria hyperborea',
        14: 'Laminaria digitata',
        18: 'Rockweed',
        38: 'Trawl track'
    }
    
    for new_idx in range(1, num_classes):
        if new_idx not in reverse_map:
            colors.append('#808080')  # Gray fallback
            continue
        
        orig_label = reverse_map[new_idx]
        class_name = class_mapping.get(orig_label, 'Unknown')
        
        color = name_to_color.get(class_name, '#808080')
        colors.append(color)
    
    print(f"[INFO] Loaded colormap with {len(colors)} colors")
    for i, (idx, color) in enumerate(zip(range(num_classes), colors)):
        if idx in reverse_map:
            orig = reverse_map[idx]
            name = class_mapping.get(orig, 'Unknown')
            print(f"  Class {idx} ({name}): {color}")
    
    return ListedColormap(colors)


def visualize_tile(x_tile, y_true=None, y_pred=None, cmap=None, idx=0):
    """
    UPDATED: Menampilkan citra tile dengan colormap yang benar
    """
    if isinstance(x_tile, torch.Tensor):
        x = x_tile.cpu().numpy()
        x = np.transpose(x, (1,2,0))
    else:
        x = x_tile

    # Pseudo-RGB
    B = x.shape[2]
    b1, b2, b3 = int(B*0.05), int(B*0.5), int(B*0.9)
    rgb = x[..., [b1, b2, b3]]
    rgb_norm = (rgb - rgb.min()) / (rgb.max() - rgb.min() + 1e-9)

    if cmap is None:
        cmap = "tab20"

    # Visualisasi
    fig, axes = plt.subplots(1, 3, figsize=(14, 4))
    
    axes[0].imshow(rgb_norm)
    axes[0].set_title("Pseudo-RGB", fontsize=11, fontweight='bold')
    axes[0].axis('off')
    
    if y_true is not None:
        im1 = axes[1].imshow(y_true, cmap=cmap, vmin=0, vmax=6)
        axes[1].set_title("Ground Truth", fontsize=11, fontweight='bold')
        axes[1].axis('off')
        cbar1 = plt.colorbar(im1, ax=axes[1], fraction=0.046, pad=0.04)
        cbar1.set_label('Class', rotation=270, labelpad=15)
    
    if y_pred is not None:
        im2 = axes[2].imshow(y_pred, cmap=cmap, vmin=0, vmax=6)
        axes[2].set_title("Prediction", fontsize=11, fontweight='bold')
        axes[2].axis('off')
        cbar2 = plt.colorbar(im2, ax=axes[2], fraction=0.046, pad=0.04)
        cbar2.set_label('Class', rotation=270, labelpad=15)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# ===========================================
# Cell 2b. [NEW] File 003 Splitting Functions
# ===========================================

def split_file_003_spatially(file_x, file_y, label_remap, 
                             train_ratio=0.6, val_ratio=0.2,
                             output_dir="../data/npy_converted_split",
                             method='auto'):
    """
    Split file 003 into train/val/test SPATIALLY
    
    Ensures that critical classes (14->4 and 18->5) appear in all splits.
    Tries horizontal split first, then vertical if needed.
    
    Args:
        file_x: Path to _x.npy (reflectance cube)
        file_y: Path to _y.npy (label mask)
        label_remap: Class remapping dict {original -> new_index}
        train_ratio: Proportion for training (default 0.6)
        val_ratio: Proportion for validation (default 0.2)
        output_dir: Where to save split files
        method: 'auto', 'horizontal', or 'vertical'
    
    Returns:
        dict: {'train': (x_path, y_path), 'val': (...), 'test': (...)}

    Raises:
        ValueError: If unable to split with all classes in all sets
    """
    
    print("\n" + "="*60)
    print("SPATIAL SPLIT OF FILE 003 (CRITICAL CLASSES 4 & 5)")
    print("="*60)
    
    # Load full image
    x_full = np.load(file_x)  # [H, W, Bands]
    y_full = np.load(file_y)  # [H, W]
    
    H, W, B = x_full.shape
    print(f"\nOriginal shape: {x_full.shape}")
    
    # Check class distribution
    unique_classes = np.unique(y_full)
    print(f"Classes in file 003: {unique_classes}")
    
    for cls in unique_classes:
        count = (y_full == cls).sum()
        pct = count / y_full.size * 100
        print(f"  Class {cls}: {count:>7} pixels ({pct:>5.2f}%)")
    
    # Helper function to check class presence
    def check_classes_in_split(y_split, split_name):
        unique = np.unique(y_split)
        has_14 = 14 in unique
        has_18 = 18 in unique
        
        if has_14 and has_18:
            print(f"  [OK] {split_name}: Classes 14 & 18 present")
        else:
            missing = []
            if not has_14:
                missing.append('14')
            if not has_18:
                missing.append('18')
            print(f"  ✗ {split_name}: Missing class {', '.join(missing)}")
        
        return has_14, has_18
    
    # Try horizontal split first
    def try_horizontal_split():
        print("\n[Trying HORIZONTAL split (left-center-right)]")
        train_end = int(W * train_ratio)
        val_end = int(W * (train_ratio + val_ratio))
        
        x_train = x_full[:, :train_end, :] # Left portion
        y_train = y_full[:, :train_end]
        
        x_val = x_full[:, train_end:val_end, :] # Center portion
        y_val = y_full[:, train_end:val_end]
        
        x_test = x_full[:, val_end:, :] # Right portion
        y_test = y_full[:, val_end:]

        print(f"  Train: columns 0-{train_end} (shape {x_train.shape})")
        print(f"  Val:   columns {train_end}-{val_end} (shape {x_val.shape})")
        print(f"  Test:  columns {val_end}-{W} (shape {x_test.shape})")
        
        # Verify all classes present
        train_ok = check_classes_in_split(y_train, "Train")
        val_ok = check_classes_in_split(y_val, "Val")
        test_ok = check_classes_in_split(y_test, "Test")

        # Success if all splits have both classes
        if all([train_ok[0], train_ok[1], val_ok[0], val_ok[1], test_ok[0], test_ok[1]]):
            print("  [OK] Horizontal split SUCCESSFUL!")
            return x_train, y_train, x_val, y_val, x_test, y_test, True
        else:
            print("  [WARNING] Horizontal split FAILED")
            return None, None, None, None, None, None, False
    
    # Try vertical split (top-center-bottom)
    def try_vertical_split():
        print("\n[Trying VERTICAL split (top-center-bottom)]")
        train_end = int(H * train_ratio)
        val_end = int(H * (train_ratio + val_ratio))
        
        x_train = x_full[:train_end, :, :] # Top portion
        y_train = y_full[:train_end, :]
        
        x_val = x_full[train_end:val_end, :, :] # Center portion
        y_val = y_full[train_end:val_end, :]
        
        x_test = x_full[val_end:, :, :] # Bottom portion
        y_test = y_full[val_end:, :]

        print(f"  Train: rows 0-{train_end} (shape {x_train.shape})")
        print(f"  Val:   rows {train_end}-{val_end} (shape {x_val.shape})")
        print(f"  Test:  rows {val_end}-{H} (shape {x_test.shape})")
        
        # Verify all classes present
        train_ok = check_classes_in_split(y_train, "Train")
        val_ok = check_classes_in_split(y_val, "Val")
        test_ok = check_classes_in_split(y_test, "Test")

        # Success if all splits have both classes
        if all([train_ok[0], train_ok[1], val_ok[0], val_ok[1], test_ok[0], test_ok[1]]):
            print("  [OK] Vertical split SUCCESSFUL!")
            return x_train, y_train, x_val, y_val, x_test, y_test, True
        else:
            print("  [WARNING] Vertical split FAILED")
            return None, None, None, None, None, None, False
    
    # Execute splitting based on method
    success = False
    
    if method == 'horizontal':
        x_train, y_train, x_val, y_val, x_test, y_test, success = try_horizontal_split()
    
    elif method == 'vertical':
        x_train, y_train, x_val, y_val, x_test, y_test, success = try_vertical_split()
    
    elif method == 'auto':
        # Try horizontal first, then vertical
        x_train, y_train, x_val, y_val, x_test, y_test, success = try_horizontal_split()
        
        if not success:
            x_train, y_train, x_val, y_val, x_test, y_test, success = try_vertical_split()
    else:
        raise ValueError(f"Invalid method: {method}. Use 'auto', 'horizontal', or 'vertical'")

    # If both method fail, raise error
    if not success:
        raise ValueError(
            "❌ CRITICAL: Unable to split file 003 automatically!\n"
            "   Classes 14 & 18 are likely clustered in one region.\n"
            "   Solutions:\n"
            "   1. Run visualize_file003.ipynb to see distribution\n"
            "   2. Adjust split ratios (try 0.5/0.25/0.25 or 0.7/0.15/0.15)\n"
            "   3. Use manual polygon selection\n"
            "   4. Accept that class 4 & 5 won't be in test (report on val only)\n"
        )
    
    # Save split files
    os.makedirs(output_dir, exist_ok=True)
    
    base_name = "massimal_smola_maholmen_202306211129-2_hsi_003"
    
    # Save train
    train_x_path = os.path.join(output_dir, f"{base_name}_train_x.npy")
    train_y_path = os.path.join(output_dir, f"{base_name}_train_y.npy")
    np.save(train_x_path, x_train)
    np.save(train_y_path, y_train)
    
    # Save val
    val_x_path = os.path.join(output_dir, f"{base_name}_val_x.npy")
    val_y_path = os.path.join(output_dir, f"{base_name}_val_y.npy")
    np.save(val_x_path, x_val)
    np.save(val_y_path, y_val)
    
    # Save test
    test_x_path = os.path.join(output_dir, f"{base_name}_test_x.npy")
    test_y_path = os.path.join(output_dir, f"{base_name}_test_y.npy")
    np.save(test_x_path, x_test)
    np.save(test_y_path, y_test)
    
    print("\n" + "="*60)
    print("SPLIT COMPLETED SUCCESSFULLY!")
    print("="*60)
    print(f"Files saved to: {output_dir}")
    print(f"  Train: {os.path.basename(train_x_path)}")
    print(f"  Val:   {os.path.basename(val_x_path)}")
    print(f"  Test:  {os.path.basename(test_x_path)}")
    
    return {
        'train': (train_x_path, train_y_path),
        'val': (val_x_path, val_y_path),
        'test': (test_x_path, test_y_path)
    }

In [ ]:
# ===========================================
# Cell 3. Dataset Loader + ADAPTIVE CLASS-BALANCED SAMPLING
# ===========================================

def load_label_mapping(json_path):
    """Membaca file label_classes.json untuk mapping id ke nama kelas"""
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    idx_to_name = {i: item["name"] for i, item in enumerate(data)}
    return idx_to_name

def normalize_reflectance(cube):
    """
    REMOVED: No additional normalization needed!
    Reflectance is already 0-1 from preprocessing
    """
    # Just ensure it's float32 and handle NaNs
    if not cube.flags.writeable:
        cube = cube.astype(np.float32, copy=True)

    if cube.dtype != np.float32:
        cube = cube.astype(np.float32, copy=False)

    np.nan_to_num(cube, copy=False)

    # NO MIN-MAX SCALING! Data is already 0-1

    return cube


class SeaweedDataset(Dataset):
    """
    IMPROVED: Dataset dengan class-balanced sampling
    """
    def __init__(self, data_files, label_map, tile_size=64, normalize=True, label_remap=None, augment=False):
        self.data_files = data_files
        self.label_map = label_map
        self.tile_size = tile_size
        self.normalize = normalize
        self.label_remap = label_remap
        self.augment = augment

        # Daftar pasangan (file_x, file_y)
        self.pairs = []
        for f in data_files:
            if f.endswith("_x.npy"):
                fy = f.replace("_x.npy", "_y.npy")
                if os.path.exists(fy):
                    self.pairs.append((f, fy))
        
        # Pre-filter tile kosong
        self.index = []
        print("[INFO] Pre-filtering empty tiles...")
        
        empty_count = 0
        valid_count = 0
        
        for file_idx, (fx, fy) in enumerate(self.pairs):
            x = np.load(fx, mmap_mode="r")
            y = np.load(fy, mmap_mode="r")
            H, W, _ = x.shape
            
            for i in range(0, H - tile_size + 1, tile_size):
                for j in range(0, W - tile_size + 1, tile_size):
                    y_tile = y[i:i+tile_size, j:j+tile_size]
                    
                    if np.any(y_tile > 0):
                        self.index.append((file_idx, i, j))
                        valid_count += 1
                    else:
                        empty_count += 1
            del x, y
            
        print(f"[INFO] Total tile valid: {valid_count}")
        print(f"[INFO] Total tile empty (filtered): {empty_count}")
        print(f"[INFO] Ratio valid/total: {valid_count/(valid_count+empty_count)*100:.2f}%")
        
        # IMPROVEMENT: Adaptive Class-balanced sampling
        if self.augment:
            print("[INFO] Applying ADAPTIVE class-balanced sampling...")
            
            # Group tiles by dominant class
            class_tiles = defaultdict(list)
            
            for idx, (file_idx, i, j) in enumerate(self.index):
                fy = self.pairs[file_idx][1]
                y = np.load(fy, mmap_mode="r")[i:i+tile_size, j:j+tile_size]
                
                # Remap
                if self.label_remap:
                    y_remap = np.zeros_like(y, dtype=np.int64)
                    for orig, new in self.label_remap.items():
                        y_remap[y == orig] = new
                    y = y_remap
                
                # Dominant class (excluding background)
                valid_mask = (y > 0)
                if valid_mask.sum() > 0:
                    dominant_class = np.bincount(y[valid_mask].flatten()).argmax()
                else:
                    dominant_class = 0
                
                class_tiles[dominant_class].append((file_idx, i, j))
            
            # IMPROVED: Adaptive target berdasarkan kelangkaan kelas.
            # Pendekatan sampling sebelumnya adalah dengan menetapkan semua kelas harus berjumlah 200 sampel seragam.
            # Pendekatan ini bersifat lebih adaptif, tidak default 200 seperti sebelumnya.
            balanced_index = []
            RARE_THRESHOLD = 100
            
            for cls in range(1, len(self.label_remap)):  # Skip background
                if cls not in class_tiles:
                    continue

                tiles = class_tiles[cls]
                n_tiles = len(tiles)


                # Adaptive target:
                # - Rare classes (<100): aggressive oversample to 300
                # - Medium classes (100-500): sample 250
                # - Common classes (>500): undersample to 200

                if n_tiles < RARE_THRESHOLD:
                    # RARE: Aggressive oversampling
                    target = min(n_tiles * 6, 400)  # 6x oversample, cap at 400
                    print(f"  Class {cls} [RARE]: {n_tiles} tiles -> {target} (6x oversample)")
                elif n_tiles < 500:
                    # MEDIUM: Moderate sampling
                    target = 250
                    print(f"  Class {cls} [MED]:  {n_tiles} tiles → {target}")
                else:
                    # COMMON: Light undersampling
                    target = 200
                    print(f"  Class {cls} [COM]:  {n_tiles} tiles → {target}")

                # Ensure minimum representation
                target = max(target, min(50, n_tiles))
                
                # Sample
                if n_tiles < target:
                    # Oversample with replacement
                    sampled_idx = np.random.choice(len(tiles), target, replace=True)
                else:
                    # Undersample
                    sampled_idx = np.random.choice(len(tiles), target, replace=False)
                
                balanced_index.extend([tiles[i] for i in sampled_idx])

            
            # Add some background tiles (20% of total)
            if 0 in class_tiles:
                bg_tiles = class_tiles[0]
                n_bg = len(balanced_index) // 5  # 20%
                n_bg = min(n_bg, len(bg_tiles))
                sampled_idx = np.random.choice(len(bg_tiles), n_bg, replace=False)
                balanced_index.extend([bg_tiles[i] for i in sampled_idx])
                print(f"  Class 0 (background): {len(bg_tiles)} tiles -> {n_bg}")
            
            self.index = balanced_index
            print(f"[INFO] After adaptive balancing: {len(self.index)} tiles")
        
        print(f"[INFO] Total tile terdaftar: {len(self.index)} dari {len(self.pairs)} file")

    def __len__(self):
        return len(self.index)

    def __getitem__(self, idx):
        file_idx, i, j = self.index[idx]
        fx, fy = self.pairs[file_idx]
        
        # Memuat tile menggunakan mmap
        x = np.load(fx, mmap_mode="r")[i:i+self.tile_size, j:j+self.tile_size, :]
        y = np.load(fy, mmap_mode="r")[i:i+self.tile_size, j:j+self.tile_size]

        if self.normalize:
            x = normalize_reflectance(x)

        # REMAP label
        if self.label_remap is not None:
            y_remap = np.zeros_like(y, dtype=np.int64)
            for orig_label, new_idx in self.label_remap.items():
                y_remap[y == orig_label] = new_idx
            y = y_remap
        else:
            y = y.astype(np.int64)

        # Konversi ke tensor
        x_tensor = torch.tensor(x.transpose(2, 0, 1), dtype=torch.float32)
        y_tensor = torch.tensor(y, dtype=torch.long)
        
        # Data Augmentation
        if self.augment:
            # Random horizontal flip
            if random.random() > 0.5:
                x_tensor = torch.flip(x_tensor, [2])
                y_tensor = torch.flip(y_tensor, [1])
            
            # Random vertical flip
            if random.random() > 0.5:
                x_tensor = torch.flip(x_tensor, [1])
                y_tensor = torch.flip(y_tensor, [0])
            
            # Random rotation 90/180/270
            k = random.choice([0, 1, 2, 3])
            if k > 0:
                x_tensor = torch.rot90(x_tensor, k, [1, 2])
                y_tensor = torch.rot90(y_tensor, k, [0, 1])
        
        return x_tensor, y_tensor


def detect_actual_classes(pairs):
    """Scan semua file y.npy untuk mendeteksi kelas yang benar-benar ada"""
    found = set()
    for _, fy in pairs:
        y = np.load(fy, mmap_mode="r")
        found |= set(np.unique(y))
    found = sorted(list(found))
    print(f"[INFO] Kelas AKTUAL yang ditemukan di dataset: {found}")
    return found

In [ ]:
# ===========================================
# Cell 4. NOVELTY 1: Spectral Self-Attention Module
# ===========================================

class SpectralAttention(nn.Module):
    """
    Spectral Self-Attention Module
    Adaptively learns importance weights for spectral channels
    """
    def __init__(self, in_channels, reduction=16):
        super().__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        
        self.fc = nn.Sequential(
            nn.Linear(in_channels, in_channels // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(in_channels // reduction, in_channels, bias=False)
        )
        
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        B, C, H, W = x.size()
        
        avg_out = self.avg_pool(x).view(B, C)
        max_out = self.max_pool(x).view(B, C)
        
        avg_weight = self.fc(avg_out)
        max_weight = self.fc(max_out)
        
        attention_weights = self.sigmoid(avg_weight + max_weight)
        
        out = x * attention_weights.view(B, C, 1, 1)
        
        return out, attention_weights

In [ ]:
# ===========================================
# Cell 5. NOVELTY 2: Multi-Objective Loss (with Focal Loss)
# ===========================================

class FocalLoss(nn.Module):
    """
    Focal Loss for addressing class imbalance
    """
    def __init__(self, alpha=None, gamma=4.0, ignore_index=0):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.ignore_index = ignore_index
    
    def forward(self, pred, target):
        ce_loss = F.cross_entropy(pred, target, weight=self.alpha, 
                                  ignore_index=self.ignore_index, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = ((1 - pt) ** self.gamma * ce_loss).mean()
        
        return focal_loss


class DiceLoss(nn.Module):
    """Dice Loss for region-level consistency"""
    def __init__(self, smooth=1.0, ignore_index=0):
        super().__init__()
        self.smooth = smooth
        self.ignore_index = ignore_index
    
    def forward(self, pred, target):
        pred = F.softmax(pred, dim=1)
        target_one_hot = F.one_hot(target, num_classes=pred.shape[1]).permute(0, 3, 1, 2).float()
        
        if self.ignore_index is not None:
            mask = (target != self.ignore_index).unsqueeze(1).float()
            pred = pred * mask
            target_one_hot = target_one_hot * mask
        
        intersection = (pred * target_one_hot).sum(dim=(2, 3))
        union = pred.sum(dim=(2, 3)) + target_one_hot.sum(dim=(2, 3))
        
        dice = (2. * intersection + self.smooth) / (union + self.smooth)
        
        return 1 - dice.mean()


class BoundaryLoss(nn.Module):
    """Boundary Loss for edge precision (DISABLED in this version)"""
    def __init__(self, ignore_index=0):
        super().__init__()
        self.ignore_index = ignore_index
    
    def forward(self, pred, target):
        # Return zero loss (disabled)
        return torch.tensor(0.0, device=pred.device, requires_grad=True)


class HybridLoss(nn.Module):
    """
    IMPROVED: Multi-Objective Hybrid Loss
    - Focal gamma increased to 5.0
    - Boundary loss disabled
    """
    def __init__(self, weight_ce=1.0, weight_dice=1.0, weight_boundary=0.0, 
                 class_weights=None, ignore_index=0, focal_gamma=5.0):
        super().__init__()
        self.weight_ce = weight_ce
        self.weight_dice = weight_dice
        self.weight_boundary = weight_boundary
        
        self.ce_loss = FocalLoss(
            alpha=class_weights, 
            gamma=focal_gamma,
            ignore_index=ignore_index
        )
        self.dice_loss = DiceLoss(ignore_index=ignore_index)
        self.boundary_loss = BoundaryLoss(ignore_index=ignore_index)
    
    def forward(self, pred, target):
        loss_ce = self.ce_loss(pred, target)
        loss_dice = self.dice_loss(pred, target)
        loss_boundary = self.boundary_loss(pred, target)
        
        total_loss = (
            self.weight_ce * loss_ce +
            self.weight_dice * loss_dice +
            self.weight_boundary * loss_boundary
        )
        
        loss_dict = {
            'total': total_loss.item(),
            'focal': loss_ce.item(),
            'dice': loss_dice.item(),
            'boundary': loss_boundary.item()
        }
        
        return total_loss, loss_dict

In [ ]:
# ===========================================
# Cell 6. Enhanced HybridSN + REDUCED Dropout
# ===========================================

class SpectralAttentionHybridSN(nn.Module):
    """
    IMPROVED: Reduced dropout from 0.3 to 0.2
    """
    def __init__(self, in_bands=300, num_classes=7, attention_reduction=16):
        super().__init__()
        
        # 3D Convolution layers
        self.conv3d_1 = nn.Conv3d(1, 16, (7,3,3), padding=(0,1,1))
        self.bn3d_1 = nn.BatchNorm3d(16)
        
        self.conv3d_2 = nn.Conv3d(16, 32, (5,3,3), padding=(0,1,1))
        self.bn3d_2 = nn.BatchNorm3d(32)
        
        self.conv3d_3 = nn.Conv3d(32, 64, (3,3,3), padding=(0,1,1))
        self.bn3d_3 = nn.BatchNorm3d(64)
        
        self.spectral_pool = nn.AdaptiveAvgPool3d((8, None, None))
        
        self.flatten_channels = 64 * 8
        
        # Spectral Attention
        self.spectral_attention = SpectralAttention(
            in_channels=self.flatten_channels,
            reduction=attention_reduction
        )
        
        # 2D Convolution layers (REDUCED DROPOUT)
        self.conv2d_1 = nn.Conv2d(self.flatten_channels, 256, 3, padding=1)
        self.bn2d_1 = nn.BatchNorm2d(256)
        self.dropout1 = nn.Dropout2d(0.2)  # REDUCED from 0.3
        
        self.conv2d_2 = nn.Conv2d(256, 128, 3, padding=1)
        self.bn2d_2 = nn.BatchNorm2d(128)
        self.dropout2 = nn.Dropout2d(0.2)  # REDUCED from 0.3
        
        self.conv2d_3 = nn.Conv2d(128, 64, 3, padding=1)
        self.bn2d_3 = nn.BatchNorm2d(64)
        
        self.classifier = nn.Conv2d(64, num_classes, 1)
        
        self.last_attention_weights = None

    def forward(self, x):
        B, Bands, H, W = x.shape
        
        # 3D CNN
        x3 = x.unsqueeze(1)
        x3 = F.relu(self.bn3d_1(self.conv3d_1(x3)))
        x3 = F.relu(self.bn3d_2(self.conv3d_2(x3)))
        x3 = F.relu(self.bn3d_3(self.conv3d_3(x3)))
        
        x3 = self.spectral_pool(x3)
        
        # Reshape
        B, C3, reduced_spec, H, W = x3.shape
        x2 = x3.view(B, C3 * reduced_spec, H, W)
        
        # Spectral Attention
        x2, attention_weights = self.spectral_attention(x2)
        self.last_attention_weights = attention_weights
        
        # 2D CNN
        x2 = self.dropout1(F.relu(self.bn2d_1(self.conv2d_1(x2))))
        x2 = self.dropout2(F.relu(self.bn2d_2(self.conv2d_2(x2))))
        x2 = F.relu(self.bn2d_3(self.conv2d_3(x2)))
        
        out = self.classifier(x2)
        
        return out
    
    def get_attention_weights(self):
        return self.last_attention_weights

In [ ]:
# ===========================================
# Cell 7. STRATIFIED Data Split (FIXED) + Separate Treatment for 003 + Tuned Class Weights
# ===========================================

data_dir = "../data/npy_converted"
split_dir = "../data/npy_converted_split"
label_json_path = "../data/annotation/segmentation_masks/label_classes.json"

label_map = load_label_mapping(label_json_path)
print(f"Jumlah total kelas di JSON: {len(label_map)}")

# ============================================
# STEP 1 : Ambil semua file kecuali file 003
# ============================================

all_x_files = sorted([os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith("_x.npy")])

file_003_name = "massimal_smola_maholmen_202306211129-2_hsi_003_processed" # ada "_processed"-nya di akhir
pairs_without_003 = []

for fx in all_x_files:
    if file_003_name not in fx:  # Exclude file 003
        fy = fx.replace("_x.npy", "_y.npy")
        if os.path.exists(fy):
            pairs_without_003.append((fx, fy))

print(f"\nTotal files (excluding 003): {len(pairs_without_003)}")


# ============================================
# STEP 2: Split File 003 Spatially
# ============================================

file_003_x = os.path.join(data_dir, f"{file_003_name}_x.npy")
file_003_y = os.path.join(data_dir, f"{file_003_name}_y.npy")

# Check if split already exists
split_exists = all([
    os.path.exists(os.path.join(split_dir, f"{file_003_name}_train_x.npy")),
    os.path.exists(os.path.join(split_dir, f"{file_003_name}_val_x.npy")),
    os.path.exists(os.path.join(split_dir, f"{file_003_name}_test_x.npy"))
])

if not split_exists:
    print("\n[INFO] Splitting file 003...")
    
    # Create temporary label_remap for splitting (will be updated later)
    temp_remap = {0:0, 8:1, 12:2, 13:3, 14:4, 18:5, 38:6}
    
    split_files_003 = split_file_003_spatially(
        file_003_x, 
        file_003_y,
        label_remap=temp_remap,
        train_ratio=0.5,
        val_ratio=0.25,
        output_dir=split_dir,
        method='auto'  # Try horizontal first, then vertical
    )
else:
    print("\n[INFO] Split files already exist, loading...")
    split_files_003 = {
        'train': (
            os.path.join(split_dir, f"{file_003_name}_train_x.npy"),
            os.path.join(split_dir, f"{file_003_name}_train_y.npy")
        ),
        'val': (
            os.path.join(split_dir, f"{file_003_name}_val_x.npy"),
            os.path.join(split_dir, f"{file_003_name}_val_y.npy")
        ),
        'test': (
            os.path.join(split_dir, f"{file_003_name}_test_x.npy"),
            os.path.join(split_dir, f"{file_003_name}_test_y.npy")
        )
    }
    print("  [OK] File 003 train split found")
    print("  [OK] File 003 val split found")
    print("  [OK] File 003 test split found")


# ============================================
# STEP 3: Create Train/Val/Test Splits
# ============================================

print("\n" + "="*60)
print("CREATING TRAIN/VAL/TEST SPLITS")
print("="*60)

#***************
# TRAIN: 10 files + file_003_train
train_pairs = [
    split_files_003['train'],
    pairs_without_003[0],    # 004
    pairs_without_003[1],    # 008
    pairs_without_003[3],    # 013
    pairs_without_003[4],    # 1228-004
    pairs_without_003[6],    # 1228-006
    pairs_without_003[8],    # 1228-012
    pairs_without_003[9],    # 1228-013
    pairs_without_003[11],   # 1228-015
    pairs_without_003[12],   # 1228-016
    pairs_without_003[16],   # 1228-021
]

# VAL: 4 files + file_003_val
val_pairs = [
    split_files_003['val'],
    pairs_without_003[2],    # 009
    pairs_without_003[7],    # 1228-009
    pairs_without_003[10],   # 1228-014
    pairs_without_003[13],   # 1228-018
]

# TEST: 2 files + file_003_test
test_pairs = [
    split_files_003['test'],
    pairs_without_003[5],    # 1228-005
    pairs_without_003[14],   # 1228-019
]


print(f"Train : {len(train_pairs)} files")
print(f"Val   : {len(val_pairs)} files")
print(f"Test  : {len(test_pairs)} files")


# ============================================
# STEP 4: Detect Classes & Create Remap
# ============================================

actual_classes = detect_actual_classes(train_pairs + val_pairs + test_pairs)
orig_classes = [int(x) for x in actual_classes]
label_remap = {orig: idx for idx, orig in enumerate(orig_classes)}
print(f"\n[INFO] Label remap: {label_remap}")

# Verify coverage
def verify_coverage(split_pairs, split_name):
    all_classes = set()
    for _, fy in split_pairs:
        y = np.load(fy, mmap_mode='r')
        all_classes |= set(np.unique(y))
    
    remapped = {label_remap[c] for c in all_classes if c in label_remap}
    print(f"\n{split_name}: {sorted(list(remapped))}")
    return remapped

train_remapped = verify_coverage(train_pairs, "TRAIN")
val_remapped = verify_coverage(val_pairs, "VAL")
test_remapped = verify_coverage(test_pairs, "TEST")

# Detect actual classes
actual_classes = detect_actual_classes(train_pairs + val_pairs + test_pairs)
orig_classes = [int(x) for x in actual_classes]
label_remap = {orig: idx for idx, orig in enumerate(orig_classes)}
print(f"\n[INFO] Label remap: {label_remap}")

# Check remapped coverage
train_remapped = {label_remap[x] for x in train_orig if x in label_remap}
val_remapped = {label_remap[x] for x in val_orig if x in label_remap}
test_remapped = {label_remap[x] for x in test_orig if x in label_remap}

# Check if class 4 & 5 (critical) are present
print(f"\n{'='*60}")
print("CRITICAL CLASS VERIFICATION")
print(f"{'='*60}")

for split_name, classes in [("Train", train_remapped), ("Val", val_remapped), ("Test", test_remapped)]:
    has_4 = 4 in classes
    has_5 = 5 in classes
    status_4 = "✅" if has_4 else "❌"
    status_5 = "✅" if has_5 else "❌"
    print(f"{split_name}: Class 4 (L. digitata) {status_4}, Class 5 (Rockweed) {status_5}")


print("="*60)

# ============================================
# STEP 5: Create Datasets
# ============================================

TILE_SIZE = 64  # Keep at 64

train_dataset = SeaweedDataset(
    [p[0] for p in train_pairs], 
    label_map, 
    tile_size=TILE_SIZE, 
    label_remap=label_remap, 
    augment=True
)

val_dataset = SeaweedDataset(
    [p[0] for p in val_pairs], 
    label_map, 
    tile_size=TILE_SIZE, 
    label_remap=label_remap, 
    augment=False
)

test_dataset = SeaweedDataset(
    [p[0] for p in test_pairs], 
    label_map, 
    tile_size=TILE_SIZE, 
    label_remap=label_remap, 
    normalize=True,  # FIXED: Enable normalization!
    augment=False
)

# ============================================
# STEP 6: TUNED Class Weights (Power 0.75)
# ============================================

print(f"\n{'='*60}")
print("COMPUTING CLASS WEIGHTS (TUNED)")
print(f"{'='*60}")

# Calculate class weights
counter = Counter()
for _, fy in train_pairs:
    y = np.load(fy, mmap_mode="r")
    for orig, new in label_remap.items():
        cnt = int((y == orig).sum())
        counter[new] += cnt

print(f"\n[INFO] Pixel counts per class: {dict(counter)}")

counts = np.array([counter.get(i, 0) for i in range(len(label_remap))], dtype=np.float64)
eps = 1e-6

# Inverse frequency
inv_freq = 1.0 / (counts + eps)
inv_freq = inv_freq / np.mean(inv_freq)

# TUNED: Power 0.75 (was 0.5 for sqrt)
class_weights_np = np.power(inv_freq, 0.75).astype(np.float32) # was np.sqrt(inv_freq).astype(np.float32)
class_weights_np[0] = 0.0

print(f"Class weights (power=0.75): {class_weights_np}")
print(f"\nWeight ratios (vs class 1):")
for i in range(1, len(class_weights_np)):
    ratio = class_weights_np[i] / class_weights_np[1]
    pixels = counter.get(i, 0)
    print(f"  Class {i}: {class_weights_np[i]:.4f} ({ratio:.2f}x) - {pixels:>8,} pixels")

weight_tensor = torch.from_numpy(class_weights_np).to(device) # untuk masuk sebagai arg value pada pemanggilan criterion HybridLoss di cell 9

num_classes_actual = len(label_remap)
print(f"\nTotal TILE train: {len(train_dataset)}, val: {len(val_dataset)}, test: {len(test_dataset)}")



In [ ]:
# ===========================================
# Cell 8. Initialize Model
# ===========================================

sample_x = np.load(train_pairs[0][0], mmap_mode="r")
in_bands_actual = sample_x.shape[2]
print(f"Band input aktual: {in_bands_actual}")

model = SpectralAttentionHybridSN(
    in_bands=in_bands_actual, 
    num_classes=num_classes_actual,
    attention_reduction=16
).to(device)

total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\n{'='*60}")
print(f"MODEL ARCHITECTURE")
print(f"{'='*60}")
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

In [ ]:
# ===========================================
# Cell 9. Setup Training (IMPROVED)
# ===========================================

import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau, LambdaLR

torch.backends.cudnn.benchmark = True

# Hyperparameters
LR = 1e-4
WEIGHT_DECAY = 1e-5
BATCH_SIZE = 1
ACCUMULATION_STEPS = 4
CLIP_NORM = 0.5

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=True)

# IMPROVED: Multi-Objective Hybrid Loss
criterion = HybridLoss(
    weight_ce=1.0,          
    weight_dice=1.0,        
    weight_boundary=0.0,    # DISABLED (was 0.1)
    class_weights=weight_tensor,
    ignore_index=0,
    focal_gamma=5.0         # INCREASED from 4.0 to 5.0
)


print("\n" + "="*60)
print("LOSS FUNCTION CONFIGURATION (IMPROVED)")
print("="*60)
print(f"  - Focal Loss: gamma=5.0 (increased from 4.0) [CHECKED]")
print(f"  - Dice Loss: weight=1.0 [CHECKED]")
print(f"  - Boundary Loss: DISABLED [CHECKED]")
print(f"  - Class weights: sqrt-smoothed [CHECKED]")

# Optimizer & Warmup Scheduler
optimizer = optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY, eps=1e-8)

# Warmup: lr ramps from lr/10 to lr over 5 epochs
def warmup_lambda(epoch):
    warmup_epochs = 5
    if epoch < warmup_epochs:
        return (epoch + 1) / warmup_epochs
    else:
        return 1.0

warmup_scheduler = LambdaLR(optimizer, lr_lambda=warmup_lambda)
main_scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=5, verbose=True, min_lr=1e-7)

print("\n" + "="*60)
print("SCHEDULER CONFIGURATION")
print("="*60)
print(f"Warmup: 5 epochs (lr: {LR/10:.2e} -> {LR:.2e})")
print(f"Main: ReduceLROnPlateau (patience=5)")

# Early Stopping
class EarlyStopping:
    def __init__(self, patience=15, min_delta=0.001, mode='max'):
        self.patience = patience
        self.min_delta = min_delta
        self.mode = mode
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.best_epoch = 0
        
    def __call__(self, current_score, epoch):
        if self.best_score is None:
            self.best_score = current_score
            self.best_epoch = epoch
            return False
        
        if self.mode == 'max':
            if current_score > self.best_score + self.min_delta:
                self.best_score = current_score
                self.counter = 0
                self.best_epoch = epoch
            else:
                self.counter += 1
        
        if self.counter >= self.patience:
            self.early_stop = True
            
        return self.early_stop

early_stopping = EarlyStopping(patience=15, min_delta=0.001, mode='max')

print("\n" + "="*60)
print("IMPROVEMENTS SUMMARY")
print("="*60)
print(f"[OK] File 003 split (class 4 & 5 in all sets)")
print(f"[OK] Stratified split (all classes in test)")
print(f"[OK] Adaptive class-balanced sampling (6x for rare)")
print(f"[OK] Focal gamma: 5.0 (was 4.0)")
print(f"[OK] Boundary loss: DISABLED")
print(f"[OK] Class weights: power 0.75 (was 0.5)")
print(f"[OK] Normalization fixed (test set enabled)")
print(f"[OK] Dropout: 0.2 (was 0.3)")
print(f"[OK] Warmup LR scheduler")
print(f"[OK] Fixed colormap loading")


# Metrics
class SegmentationMetrics:
    def __init__(self, num_classes, ignore_index=0):
        self.num_classes = num_classes
        self.ignore_index = ignore_index
        self.reset()
    
    def reset(self):
        self.total_intersection = torch.zeros(self.num_classes)
        self.total_union = torch.zeros(self.num_classes)
        self.total_correct = 0
        self.total_pixels = 0
    
    def update(self, pred, target):
        valid = (target != self.ignore_index)
        pred = pred[valid]
        target = target[valid]
        
        self.total_correct += (pred == target).sum().item()
        self.total_pixels += valid.sum().item()
        
        for cls in range(self.num_classes):
            pred_i = (pred == cls)
            target_i = (target == cls)
            intersection = (pred_i & target_i).sum().item()
            union = (pred_i | target_i).sum().item()
            
            self.total_intersection[cls] += intersection
            self.total_union[cls] += union
    
    def get_metrics(self):
        pixel_acc = self.total_correct / (self.total_pixels + 1e-9)
        
        iou_per_class = self.total_intersection / (self.total_union + 1e-9)
        valid_ious = []
        for i in range(1, self.num_classes):
            if self.total_union[i] > 0:
                valid_ious.append(iou_per_class[i].item())
        
        mean_iou = np.mean(valid_ious) if valid_ious else 0.0
        
        return pixel_acc, mean_iou, iou_per_class.numpy()

In [ ]:
# ===========================================
# Cell 10. Training Loop (IMPROVED)
# ===========================================

from tqdm import tqdm
import time

START_EPOCH = 1
NUM_EPOCHS = 50
best_val_miou = 0.0

checkpoint_path = "hybridsn_sgmt_ver5_FIXED_checkpoint.pth"
best_model_path = "hybridsn_sgmt_ver5_FIXED_best_model.pth"

if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint["model_state"])
    optimizer.load_state_dict(checkpoint["optimizer_state"])
    START_EPOCH = checkpoint["epoch"] + 1
    best_val_miou = checkpoint.get("best_val_miou", 0.0)
    
    if "early_stopping_state" in checkpoint:
        early_stopping.counter = checkpoint["early_stopping_state"]["counter"]
        early_stopping.best_score = checkpoint["early_stopping_state"]["best_score"]
        early_stopping.best_epoch = checkpoint["early_stopping_state"]["best_epoch"]
    
    print(f"[INFO] Resume dari epoch {START_EPOCH}")
else:
    print("[INFO] Training dari awal")

history = {
    "train_loss": [], "val_loss": [], 
    "train_acc": [], "val_acc": [], 
    "val_miou": [],
    "train_loss_focal": [], "train_loss_dice": [], "train_loss_boundary": [],
    "val_loss_focal": [], "val_loss_dice": [], "val_loss_boundary": []
}

def train_one_epoch(model, loader, criterion, optimizer, metrics, device):
    model.train()
    running_loss = 0.0
    running_loss_components = {'focal': 0.0, 'dice': 0.0, 'boundary': 0.0}
    metrics.reset()
    
    pbar = tqdm(loader, desc="Training", leave=False)
    optimizer.zero_grad()

    nan_count = 0
    valid_batches = 0
    
    for i, (xb, yb) in enumerate(pbar):
        xb = xb.to(device, non_blocking=True)
        yb = yb.to(device, non_blocking=True)

        if torch.all(yb == 0):
            continue
        
        logits = model(xb)
        loss, loss_dict = criterion(logits, yb)
        loss = loss / ACCUMULATION_STEPS

        if torch.isnan(loss) or torch.isinf(loss):
            nan_count += 1
            continue
        
        loss.backward()
        
        if (i + 1) % ACCUMULATION_STEPS == 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), CLIP_NORM)
            optimizer.step()
            optimizer.zero_grad()
        
        running_loss += loss.item() * ACCUMULATION_STEPS * xb.size(0)
        running_loss_components['focal'] += loss_dict['focal'] * xb.size(0)
        running_loss_components['dice'] += loss_dict['dice'] * xb.size(0)
        running_loss_components['boundary'] += loss_dict['boundary'] * xb.size(0)
        valid_batches += xb.size(0)
        
        preds = logits.argmax(dim=1)
        metrics.update(preds, yb)
        
        pbar.set_postfix({
            "loss": f"{loss.item()*ACCUMULATION_STEPS:.4f}",
            "focal": f"{loss_dict['focal']:.3f}",
            "dice": f"{loss_dict['dice']:.3f}"
        })
        
    if nan_count > 0:
        print(f"  [INFO] Skipped {nan_count} NaN batches")
    
    avg_loss = running_loss / valid_batches if valid_batches > 0 else 0.0
    avg_loss_focal = running_loss_components['focal'] / valid_batches if valid_batches > 0 else 0.0
    avg_loss_dice = running_loss_components['dice'] / valid_batches if valid_batches > 0 else 0.0
    avg_loss_boundary = running_loss_components['boundary'] / valid_batches if valid_batches > 0 else 0.0
    
    pixel_acc, _, _ = metrics.get_metrics()
    
    return avg_loss, pixel_acc, avg_loss_focal, avg_loss_dice, avg_loss_boundary

def validate(model, loader, criterion, metrics, device):
    model.eval()
    running_loss = 0.0
    running_loss_components = {'focal': 0.0, 'dice': 0.0, 'boundary': 0.0}
    valid_batches = 0
    metrics.reset()

    nan_count = 0
    
    with torch.no_grad():
        for xb, yb in tqdm(loader, desc="Validation", leave=False):
            xb = xb.to(device, non_blocking=True)
            yb = yb.to(device, non_blocking=True)

            if torch.all(yb == 0):
                continue
            
            logits = model(xb)
            loss, loss_dict = criterion(logits, yb)

            if torch.isnan(loss) or torch.isinf(loss):
                nan_count += 1
                continue
            
            running_loss += loss.item() * xb.size(0)
            running_loss_components['focal'] += loss_dict['focal'] * xb.size(0)
            running_loss_components['dice'] += loss_dict['dice'] * xb.size(0)
            running_loss_components['boundary'] += loss_dict['boundary'] * xb.size(0)
            valid_batches += xb.size(0)
            
            preds = logits.argmax(dim=1)
            metrics.update(preds, yb)

    if nan_count > 0:
        print(f"  [INFO] Skipped {nan_count} NaN batches")
    
    if valid_batches > 0:
        avg_loss = running_loss / valid_batches
        avg_loss_focal = running_loss_components['focal'] / valid_batches
        avg_loss_dice = running_loss_components['dice'] / valid_batches
        avg_loss_boundary = running_loss_components['boundary'] / valid_batches
    else:
        avg_loss = float('nan')
        avg_loss_focal = avg_loss_dice = avg_loss_boundary = float('nan')
    
    pixel_acc, mean_iou, iou_per_class = metrics.get_metrics()
    
    return avg_loss, pixel_acc, mean_iou, iou_per_class, avg_loss_focal, avg_loss_dice, avg_loss_boundary


# Training loop
print("\n" + "="*60)
print("STARTING TRAINING (IMPROVED VERSION)")
print("="*60)
print(f"Max epochs: {NUM_EPOCHS}")

stopped_early = False

for epoch in range(START_EPOCH, NUM_EPOCHS + 1):
    print(f"\n{'='*60}")
    print(f"Epoch {epoch}/{NUM_EPOCHS}")
    print(f"{'='*60}")
    
    start_time = time.time()
    
    if torch.cuda.is_available():
        print(f"GPU Memory: {torch.cuda.memory_allocated()/1e9:.2f} GB allocated")
    
    # Training
    train_metrics = SegmentationMetrics(num_classes_actual, ignore_index=0)
    train_loss, train_acc, train_focal, train_dice, train_bnd = train_one_epoch(
        model, train_loader, criterion, optimizer, train_metrics, device
    )
    
    # Validation
    val_metrics = SegmentationMetrics(num_classes_actual, ignore_index=0)
    val_loss, val_acc, val_miou, val_iou_per_class, val_focal, val_dice, val_bnd = validate(
        model, val_loader, criterion, val_metrics, device
    )
    
    # Scheduler step (WARMUP AWARE)
    if epoch <= 5:
        warmup_scheduler.step()
        print(f"  [Warmup] LR: {optimizer.param_groups[0]['lr']:.2e}")
    else:
        main_scheduler.step(val_miou)
    
    # Logging
    elapsed = time.time() - start_time
    print(f"\nResults:")
    print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
    print(f"    ├─ Focal: {train_focal:.4f} | Dice: {train_dice:.4f}")
    print(f"  Val Loss  : {val_loss:.4f} | Val Acc  : {val_acc:.4f}")
    print(f"    ├─ Focal: {val_focal:.4f} | Dice: {val_dice:.4f}")
    print(f"  Val mIoU  : {val_miou:.4f}")
    print(f"  Time      : {elapsed/60:.2f} min")
    print(f"  IoU per class: {val_iou_per_class[1:]}")
    
    # Save history
    history["train_loss"].append(train_loss)
    history["val_loss"].append(val_loss)
    history["train_acc"].append(train_acc)
    history["val_acc"].append(val_acc)
    history["val_miou"].append(val_miou)
    history["train_loss_focal"].append(train_focal)
    history["train_loss_dice"].append(train_dice)
    history["train_loss_boundary"].append(train_bnd)
    history["val_loss_focal"].append(val_focal)
    history["val_loss_dice"].append(val_dice)
    history["val_loss_boundary"].append(val_bnd)
    
    # Checkpoint
    checkpoint = {
        "epoch": epoch,
        "model_state": model.state_dict(),
        "optimizer_state": optimizer.state_dict(),
        "warmup_scheduler_state": warmup_scheduler.state_dict(),
        "main_scheduler_state": main_scheduler.state_dict(),
        "best_val_miou": best_val_miou,
        "history": history,
        "early_stopping_state": {
            "counter": early_stopping.counter,
            "best_score": early_stopping.best_score,
            "best_epoch": early_stopping.best_epoch
        }
    }
    torch.save(checkpoint, checkpoint_path)
    
    # Save best model
    if val_miou > best_val_miou:
        best_val_miou = val_miou
        torch.save(checkpoint, best_model_path)
        print(f"[OK] Best model saved! (mIoU: {best_val_miou:.4f})")
    
    # Early Stopping
    if early_stopping(val_miou, epoch):
        print(f"\n{'='*60}")
        print(f"EARLY STOPPING TRIGGERED!")
        print(f"{'='*60}")
        print(f"Best mIoU: {early_stopping.best_score:.4f} at epoch {early_stopping.best_epoch}")
        stopped_early = True
        break
    else:
        if early_stopping.counter > 0:
            print(f"[Early Stop] No improvement: {early_stopping.counter}/{early_stopping.patience}")
    
    torch.cuda.empty_cache()

print("\n" + "="*60)
if stopped_early:
    print("Training stopped early!")
else:
    print("Training completed!")
print(f"Best validation mIoU: {best_val_miou:.4f}")
print("="*60)

In [ ]:
# Load colormap ONCE for all visualizations
global_cmap = load_colormap_from_json(label_json_path, label_remap, num_classes_actual)

In [ ]:
# ===========================================
# Cell 11. Plot Training History (Enhanced)
# ===========================================

fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# Loss
axes[0, 0].plot(history["train_loss"], label="Train Loss", marker='o')
axes[0, 0].plot(history["val_loss"], label="Val Loss", marker='s')
axes[0, 0].set_xlabel("Epoch")
axes[0, 0].set_ylabel("Loss")
axes[0, 0].set_title("Training vs Validation Loss")
axes[0, 0].legend()
axes[0, 0].grid(True)

# Accuracy
axes[0, 1].plot(history["train_acc"], label="Train Acc", marker='o')
axes[0, 1].plot(history["val_acc"], label="Val Acc", marker='s')
axes[0, 1].set_xlabel("Epoch")
axes[0, 1].set_ylabel("Pixel Accuracy")
axes[0, 1].set_title("Training vs Validation Accuracy")
axes[0, 1].legend()
axes[0, 1].grid(True)

# mIoU
axes[0, 2].plot(history["val_miou"], label="Val mIoU", marker='d', color='green')
axes[0, 2].set_xlabel("Epoch")
axes[0, 2].set_ylabel("Mean IoU")
axes[0, 2].set_title("Validation mIoU")
axes[0, 2].legend()
axes[0, 2].grid(True)

# Loss Components - Training
axes[1, 0].plot(history["train_loss_focal"], label="Focal Loss", marker='o')
axes[1, 0].plot(history["train_loss_dice"], label="Dice Loss", marker='s')
axes[1, 0].plot(history["train_loss_boundary"], label="Boundary Loss", marker='^')
axes[1, 0].set_xlabel("Epoch")
axes[1, 0].set_ylabel("Loss Value")
axes[1, 0].set_title("Training Loss Components")
axes[1, 0].legend()
axes[1, 0].grid(True)

# Loss Components - Validation
axes[1, 1].plot(history["val_loss_focal"], label="Focal Loss", marker='o')
axes[1, 1].plot(history["val_loss_dice"], label="Dice Loss", marker='s')
axes[1, 1].plot(history["val_loss_boundary"], label="Boundary Loss", marker='^')
axes[1, 1].set_xlabel("Epoch")
axes[1, 1].set_ylabel("Loss Value")
axes[1, 1].set_title("Validation Loss Components")
axes[1, 1].legend()
axes[1, 1].grid(True)

# Loss ratio analysis
if len(history["val_loss_focal"]) > 0:
    total = np.array(history["val_loss_focal"]) + np.array(history["val_loss_dice"]) + np.array(history["val_loss_boundary"])
    focal_ratio = np.array(history["val_loss_focal"]) / (total + 1e-9)
    dice_ratio = np.array(history["val_loss_dice"]) / (total + 1e-9)
    bnd_ratio = np.array(history["val_loss_boundary"]) / (total + 1e-9)
    
    axes[1, 2].plot(focal_ratio, label="Focal Ratio", marker='o')
    axes[1, 2].plot(dice_ratio, label="Dice Ratio", marker='s')
    axes[1, 2].plot(bnd_ratio, label="Boundary Ratio", marker='^')
    axes[1, 2].set_xlabel("Epoch")
    axes[1, 2].set_ylabel("Proportion")
    axes[1, 2].set_title("Loss Component Ratios (Validation)")
    axes[1, 2].legend()
    axes[1, 2].grid(True)

plt.tight_layout()
plt.savefig("training_history_ver5_enhanced.png", dpi=150)
plt.show()

In [ ]:
# ===========================================
# Cell 12. Spectral Attention Visualization
# ===========================================

def visualize_spectral_attention(model, loader, num_samples=3):
    """Visualize learned spectral attention weights"""
    model.eval()
    
    attention_weights_list = []
    sample_images = []
    
    with torch.no_grad():
        for i, (xb, yb) in enumerate(loader):
            if i >= num_samples:
                break
            
            xb = xb.to(device)
            _ = model(xb)  # Forward pass
            
            # Get attention weights
            attn_weights = model.get_attention_weights()
            if attn_weights is not None:
                attention_weights_list.append(attn_weights[0].cpu().numpy())
                sample_images.append(xb[0])
    
    if not attention_weights_list:
        print("No attention weights captured!")
        return
    
    # Plot attention weights
    fig, axes = plt.subplots(num_samples, 2, figsize=(14, 4*num_samples))
    
    for i in range(num_samples):
        # Attention weights
        attn = attention_weights_list[i]
        axes[i, 0].bar(range(len(attn)), attn, color='steelblue', alpha=0.7)
        axes[i, 0].set_xlabel("Spectral Channel (after 3D CNN)")
        axes[i, 0].set_ylabel("Attention Weight")
        axes[i, 0].set_title(f"Sample {i+1}: Learned Spectral Attention")
        axes[i, 0].grid(True, alpha=0.3)
        
        # Top-k important channels
        top_k = 10
        top_indices = np.argsort(attn)[-top_k:]
        axes[i, 0].bar(top_indices, attn[top_indices], color='red', alpha=0.7, 
                       label=f'Top-{top_k} channels')
        axes[i, 0].legend()
        
        # Original hyperspectral signature (averaged)
        img = sample_images[i].cpu().numpy()  # [Bands, H, W]
        mean_spectrum = img.mean(axis=(1, 2))
        
        axes[i, 1].plot(mean_spectrum, color='darkgreen', linewidth=1.5)
        axes[i, 1].set_xlabel("Original Spectral Band")
        axes[i, 1].set_ylabel("Mean Reflectance")
        axes[i, 1].set_title(f"Sample {i+1}: Mean Spectral Signature")
        axes[i, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig("spectral_attention_weights_ver5.png", dpi=150)
    plt.show()
    
    # Print statistics
    print("\n" + "="*60)
    print("SPECTRAL ATTENTION ANALYSIS")
    print("="*60)
    for i, attn in enumerate(attention_weights_list):
        top_5_idx = np.argsort(attn)[-5:]
        print(f"\nSample {i+1}:")
        print(f"  Mean attention: {attn.mean():.4f}")
        print(f"  Std attention: {attn.std():.4f}")
        print(f"  Max attention: {attn.max():.4f}")
        print(f"  Top-5 channels: {top_5_idx} with weights {attn[top_5_idx]}")

# Visualize attention on validation set
print("\nVisualizing spectral attention weights...")
visualize_spectral_attention(model, val_loader, num_samples=3)

In [ ]:
# ===========================================
# Cell 13. Testing dan Visualisasi Output Segmentation
# ===========================================

# Load best model
best_checkpoint = torch.load(best_model_path, map_location=device)
model.load_state_dict(best_checkpoint["model_state"])
print(f"Loaded best model from epoch {best_checkpoint['epoch']}")

# Test evaluation
test_metrics = SegmentationMetrics(num_classes_actual, ignore_index=0)
test_loss, test_acc, test_miou, test_iou_per_class, test_focal, test_dice, test_bnd = validate(
    model, test_loader, criterion, test_metrics, device
)

print("\n" + "="*60)
print("TEST RESULTS")
print("="*60)
print(f"Test Loss     : {test_loss:.4f}")
print(f"  ├─ Focal Loss  : {test_focal:.4f}") # rename ce menjadi focal 
print(f"  ├─ Dice Loss: {test_dice:.4f}")
print(f"  └─ Bnd Loss : {test_bnd:.4f}")
print(f"Test Accuracy : {test_acc:.4f}")
print(f"Test mIoU     : {test_miou:.4f}")
print(f"IoU per class : {test_iou_per_class[1:]}")

# Visualisasi beberapa prediksi SEGMENTATION MAP
model.eval()
num_vis = 5
vis_samples = []

print("\n" + "="*60)
print("GENERATING SEGMENTATION MAPS")
print("="*60)

with torch.no_grad():
    for i, (xb, yb) in enumerate(test_loader):
        if i >= num_vis:
            break
        xb = xb.to(device)
        logits = model(xb)
        preds = logits.argmax(dim=1)  # [B, H, W] - SEGMENTATION MAP
        
        vis_samples.append((xb[0], yb[0], preds[0]))
        print(f"Sample {i+1}: Input shape {xb.shape}, Output shape {preds.shape}")

# Plot visualisasi segmentation maps
print("\nGenerating visualizations...")
for i, (x, y_true, y_pred) in enumerate(vis_samples):
    print(f"Visualizing sample {i+1}/{num_vis}")
    visualize_tile(x, y_true.cpu().numpy(), y_pred.cpu().numpy(), 
                   cmap=global_cmap, idx=i)

print("\n[OK] Output: SEGMENTATION MAPS")
print("  Format: [H, W] dengan setiap pixel berisi class label (0-6)")
print("  Visualisasi: Setiap warna merepresentasikan habitat class berbeda")

In [ ]:
# ===========================================
# Cell 14. Confusion Matrix
# ===========================================

from sklearn.metrics import confusion_matrix
import seaborn as sns

# Collect predictions untuk confusion matrix
all_preds = []
all_targets = []

model.eval()
with torch.no_grad():
    for xb, yb in tqdm(test_loader, desc="Computing CM"):
        xb = xb.to(device)
        logits = model(xb)
        preds = logits.argmax(dim=1)
        
        # Flatten dan filter valid pixels
        preds_flat = preds.cpu().numpy().flatten()
        targets_flat = yb.numpy().flatten()
        
        valid = targets_flat != 0  # Exclude background
        all_preds.extend(preds_flat[valid])
        all_targets.extend(targets_flat[valid])

# Compute confusion matrix
cm = confusion_matrix(all_targets, all_preds, labels=list(range(1, num_classes_actual)))

# Plot
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=range(1, num_classes_actual),
            yticklabels=range(1, num_classes_actual))
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix (Test Set - Excluding Background)')
plt.tight_layout()
plt.savefig('confusion_matrix_ver5.png', dpi=150)
plt.show()

print("Confusion matrix saved!")

In [ ]:
# ===========================================
# Cell 15. Full Image Segmentation with Sliding Window
# ===========================================

def segment_full_image(model, image_path, mask_path=None, tile_size=64, stride=32, device='cuda'):
    """
    Segmentasi full image dengan sliding window approach
    
    Args:
        model: trained segmentation model
        image_path: path ke file _x.npy
        mask_path: path ke file _y.npy (optional, untuk comparison)
        tile_size: ukuran tile untuk inference
        stride: step size untuk sliding window (stride < tile_size untuk smooth overlap)
        device: 'cuda' or 'cpu'
    
    Returns:
        prediction_map: segmentation result [H, W]
        full_image: original image [H, W, Bands]
        ground_truth: GT mask if mask_path provided, else None
    """
    model.eval()
    
    print(f"[INFO] Loading full image from: {image_path}")
    
    # Load full image
    x_full = np.load(image_path, mmap_mode='r')  # [H, W, Bands]
    H, W, B = x_full.shape
    print(f"[INFO] Image shape: {x_full.shape}")
    
    # Normalize
    x_full_norm = normalize_reflectance(x_full.copy())
    
    # Prepare output with soft voting
    prediction_accumulator = np.zeros((num_classes_actual, H, W), dtype=np.float32)
    count_map = np.zeros((H, W), dtype=np.float32)
    
    # Calculate number of tiles
    n_tiles_h = (H - tile_size) // stride + 1
    n_tiles_w = (W - tile_size) // stride + 1
    total_tiles = n_tiles_h * n_tiles_w
    
    print(f"[INFO] Processing {total_tiles} tiles ({n_tiles_h}x{n_tiles_w})...")
    
    with torch.no_grad():
        tile_count = 0
        for i in range(0, H - tile_size + 1, stride):
            for j in range(0, W - tile_size + 1, stride):
                # Extract tile
                tile = x_full_norm[i:i+tile_size, j:j+tile_size, :]
                tile_tensor = torch.from_numpy(tile.transpose(2,0,1)).unsqueeze(0).float().to(device)
                
                # Predict
                logits = model(tile_tensor)  # [1, C, H, W]
                probs = F.softmax(logits, dim=1).squeeze(0).cpu().numpy()  # [C, H, W]
                
                # Accumulate predictions (soft voting)
                prediction_accumulator[:, i:i+tile_size, j:j+tile_size] += probs
                count_map[i:i+tile_size, j:j+tile_size] += 1
                
                tile_count += 1
                if tile_count % 100 == 0:
                    print(f"  Processed {tile_count}/{total_tiles} tiles ({tile_count/total_tiles*100:.1f}%)")
    
    # Average overlapping predictions and get argmax
    prediction_accumulator /= (count_map[np.newaxis, :, :] + 1e-9)
    final_pred = prediction_accumulator.argmax(axis=0).astype(np.int32)
    
    print(f"[INFO] Segmentation complete!")
    print(f"[INFO] Unique classes predicted: {np.unique(final_pred)}")
    
    # Load ground truth if provided
    ground_truth = None
    if mask_path and os.path.exists(mask_path):
        ground_truth = np.load(mask_path)
        print(f"[INFO] Ground truth loaded. Unique classes: {np.unique(ground_truth)}")
        
        # Remap ground truth
        if label_remap is not None:
            gt_remap = np.zeros_like(ground_truth, dtype=np.int32)
            for orig_label, new_idx in label_remap.items():
                gt_remap[ground_truth == orig_label] = new_idx
            ground_truth = gt_remap
    
    return final_pred, x_full, ground_truth


# Visualize full image segmentation
print("\n" + "="*60)
print("FULL IMAGE SEGMENTATION")
print("="*60)

# Select a test image
test_file_idx = 0  # Change this to visualize different images
test_file_x = test_pairs[test_file_idx][0]
test_file_y = test_pairs[test_file_idx][1]

print(f"Selected image: {os.path.basename(test_file_x)}")

# Perform segmentation
full_pred, full_img, full_gt = segment_full_image(
    model, 
    test_file_x, 
    test_file_y,
    tile_size=TILE_SIZE, 
    stride=TILE_SIZE // 2,  # 50% overlap
    device=device
)

# Calculate metrics for full image
if full_gt is not None:
    valid_mask = (full_gt != 0)
    if valid_mask.sum() > 0:
        valid_pred = full_pred[valid_mask]
        valid_gt = full_gt[valid_mask]
        
        full_acc = (valid_pred == valid_gt).sum() / len(valid_pred)
        
        # Calculate IoU per class
        full_iou_per_class = []
        for cls in range(1, num_classes_actual):
            pred_i = (valid_pred == cls)
            gt_i = (valid_gt == cls)
            intersection = (pred_i & gt_i).sum()
            union = (pred_i | gt_i).sum()
            iou = intersection / (union + 1e-9) if union > 0 else 0.0
            full_iou_per_class.append(iou)
        
        full_miou = np.mean([iou for iou in full_iou_per_class if iou > 0])
        
        print(f"\n[METRICS] Full Image:")
        print(f"  Pixel Accuracy: {full_acc:.4f}")
        print(f"  Mean IoU: {full_miou:.4f}")
        print(f"  IoU per class: {full_iou_per_class}")

# Visualize
fig, axes = plt.subplots(1, 3, figsize=(20, 6))

# Pseudo-RGB
rgb_idx = [int(full_img.shape[2]*0.05), int(full_img.shape[2]*0.5), int(full_img.shape[2]*0.9)]
rgb = full_img[..., rgb_idx]
rgb_norm = (rgb - rgb.min()) / (rgb.max() - rgb.min() + 1e-9)

# Load colormap
if os.path.exists(label_json_path):
    with open(label_json_path, "r") as f:
        label_info = json.load(f)
    custom_colors = [c["color"][:7] for c in label_info]
    cmap = ListedColormap(custom_colors)
else:
    cmap = "tab20"

axes[0].imshow(rgb_norm)
axes[0].set_title(f"Full Image (Pseudo-RGB)\nShape: {full_img.shape[:2]}", fontsize=12)
axes[0].axis('off')

if full_gt is not None:
    axes[1].imshow(full_gt, cmap=cmap, vmin=0, vmax=num_classes_actual-1)
    axes[1].set_title("Ground Truth Segmentation", fontsize=12)
    axes[1].axis('off')

axes[2].imshow(full_pred, cmap=cmap, vmin=0, vmax=num_classes_actual-1)
axes[2].set_title(f"Predicted Segmentation\nmIoU: {full_miou:.3f}" if full_gt is not None else "Predicted Segmentation", fontsize=12)
axes[2].axis('off')

plt.tight_layout()
plt.savefig('full_image_segmentation_ver5.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\n[OK] Full image segmentation saved!")
print(f"  Resolution: {full_pred.shape}")
print(f"  File: full_image_segmentation_ver5.png")


# Optional: Visualize multiple test images
print("\n" + "="*60)
print("BATCH FULL IMAGE SEGMENTATION (ALL TEST IMAGES)")
print("="*60)

for idx, (fx, fy) in enumerate(test_pairs):
    print(f"\nProcessing image {idx+1}/{len(test_pairs)}: {os.path.basename(fx)}")
    
    pred, img, gt = segment_full_image(
        model, fx, fy,
        tile_size=TILE_SIZE,
        stride=TILE_SIZE // 2,
        device=device
    )
    
    # Create visualization
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    rgb_idx = [int(img.shape[2]*0.05), int(img.shape[2]*0.5), int(img.shape[2]*0.9)]
    rgb = img[..., rgb_idx]
    rgb_norm = (rgb - rgb.min()) / (rgb.max() - rgb.min() + 1e-9)
    
    axes[0].imshow(rgb_norm)
    axes[0].set_title(f"Image {idx+1}: Pseudo-RGB")
    axes[0].axis('off')
    
    if gt is not None:
        axes[1].imshow(gt, cmap=cmap)
        axes[1].set_title("Ground Truth")
        axes[1].axis('off')
    
    axes[2].imshow(pred, cmap=cmap)
    axes[2].set_title("Prediction")
    axes[2].axis('off')
    
    plt.tight_layout()
    plt.savefig(f'full_image_seg_test_{idx+1}.png', dpi=120, bbox_inches='tight')
    plt.close()
    
    print(f"  [OK] Saved: full_image_seg_test_{idx+1}.png")

print("\n[OK] All test images processed!")
print(f"[OK] Generated {len(test_pairs)} full segmentation maps")


In [ ]:
# ===========================================
# Cell 15b. Dataset Distribution Analysis (untuk solusi test set issue)
# ===========================================

print("\n" + "="*60)
print("DATASET CLASS DISTRIBUTION ANALYSIS")
print("="*60)

def analyze_file_distribution(pairs, split_name):
    """Analisis distribusi kelas per file"""
    print(f"\n{split_name} Split Analysis:")
    print(f"{'='*50}")
    
    file_class_info = []
    
    for idx, (fx, fy) in enumerate(pairs):
        y = np.load(fy, mmap_mode='r')
        unique, counts = np.unique(y, return_counts=True)
        
        # Convert to dict
        class_dist = dict(zip(unique.tolist(), counts.tolist()))
        
        # Get remapped classes
        remapped_dist = {}
        for orig, cnt in class_dist.items():
            if orig in label_remap:
                new_idx = label_remap[orig]
                remapped_dist[new_idx] = cnt
        
        file_class_info.append({
            'file': os.path.basename(fx),
            'distribution': remapped_dist,
            'classes_present': sorted(remapped_dist.keys())
        })
        
        print(f"\nFile {idx+1}: {os.path.basename(fx)}")
        print(f"  Classes present: {sorted(remapped_dist.keys())}")
        print(f"  Pixel counts: {remapped_dist}")
    
    # Summary
    all_classes = set()
    for info in file_class_info:
        all_classes.update(info['classes_present'])
    
    print(f"\n{split_name} Summary:")
    print(f"  Total files: {len(pairs)}")
    print(f"  Classes covered: {sorted(all_classes)}")
    print(f"  Missing classes: {set(range(num_classes_actual)) - all_classes}")
    
    return file_class_info

# Analyze all splits
train_analysis = analyze_file_distribution(train_pairs, "TRAIN")
val_analysis = analyze_file_distribution(val_pairs, "VALIDATION")
test_analysis = analyze_file_distribution(test_pairs, "TEST")

# Check coverage
print("\n" + "="*60)
print("OVERALL COVERAGE CHECK")
print("="*60)

all_train_classes = set()
all_val_classes = set()
all_test_classes = set()

for info in train_analysis:
    all_train_classes.update(info['classes_present'])
for info in val_analysis:
    all_val_classes.update(info['classes_present'])
for info in test_analysis:
    all_test_classes.update(info['classes_present'])

print(f"\nClasses in Train: {sorted(all_train_classes)}")
print(f"Classes in Val:   {sorted(all_val_classes)}")
print(f"Classes in Test:  {sorted(all_test_classes)}")

missing_in_test = all_train_classes - all_test_classes
if missing_in_test:
    print(f"\n [WARNING]: Classes {sorted(missing_in_test)} are in TRAIN but NOT in TEST!")
    print(f"   This means we cannot evaluate model performance on these classes.")
    
    # Suggest rebalancing
    print(f"\n💡 RECOMMENDATION:")
    print(f"   Option 1: Move files from train/val to test that contain classes {sorted(missing_in_test)}")
    
    # Find files in train that have missing classes
    candidate_files = []
    for info in train_analysis:
        if missing_in_test & set(info['classes_present']):
            candidate_files.append((info['file'], info['classes_present']))
    
    if candidate_files:
        print(f"\n   Candidate files to move to test set:")
        for fname, classes in candidate_files[:3]:  # Show top 3
            print(f"     - {fname}: has classes {sorted(classes)}")
else:
    print(f"\n✓ All classes are represented in test set!")

In [ ]:
# ===========================================
# Cell 17. Save Results Summary
# ===========================================

results_summary = {
    "model": "SpectralAttentionHybridSN",
    "version": "v4_improved",
    "novelties": ["Spectral Self-Attention", "Multi-Objective Loss (Focal+Dice+Boundary)"],
    "improvements": [
        "Learning rate: 5e-6 → 1e-4",
        "Tile size: 32 → 64",
        "Boundary loss weight: 0.5 → 0.1",
        "Data augmentation: flip + rotation",
        "Focal loss replacing CE",
        "Early stopping patience: 10 → 15"
    ],
    "hyperparameters": {
        "learning_rate": LR,
        "batch_size": BATCH_SIZE,
        "accumulation_steps": ACCUMULATION_STEPS,
        "tile_size": TILE_SIZE,
        "num_epochs": NUM_EPOCHS,
        "attention_reduction": 16,
        "loss_weights": {"focal": 1.0, "dice": 1.0, "boundary": 0.1},
        "focal_gamma": 2.0,
        "early_stopping_patience": 15
    },
    "test_results": {
        "pixel_accuracy": float(test_acc),
        "mean_iou": float(test_miou),
        "loss_total": float(test_loss),
        "loss_focal": float(test_focal),
        "loss_dice": float(test_dice),
        "loss_boundary": float(test_bnd),
        "iou_per_class": test_iou_per_class.tolist()
    },
    "training_history": history
}

# Save to JSON
with open("results_ver5_summary.json", "w") as f:
    json.dump(results_summary, f, indent=2)

print("\n" + "="*60)
print("EXPERIMENT SUMMARY")
print("="*60)
print(json.dumps(results_summary["test_results"], indent=2))
print("\nResults saved to: results_ver5_summary.json")
print("\n[OK]Training complete!")